# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [37]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.optim as optim

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [38]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [39]:
class SimpleNet(nn.Module):
# TODO:define model
# 使用LeNet
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2), 
            #(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
            nn.ReLU(),      #input_size=(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),#output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),      #input_size=(16*10*10)
            nn.MaxPool2d(2, 2)  #output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)

   
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
       
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x



    
model = SimpleNet()
model.to("cpu")

Learningrate = 0.001  
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=Learningrate, momentum=0.9)

Next, we can start to train and evaluate!

In [43]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    running_loss=0.
    running_acc=0.   
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()  
        output=model(images) 
        loss=criterion(output,labels) 
        loss.backward()    
        optimizer.step()   
 
        running_loss+=loss.item()
      
        valu,predict=torch.max(output,1)
        correct_num=(predict==labels).sum()
        running_acc+=correct_num.item()
    running_loss/=len(train_dataset)
    running_acc/=len(train_dataset)
    print("[%d/%d] Loss: %.5f, Acc: %.2f"%(epoch+1,NUM_EPOCHS,running_loss,
                                           100*running_acc))
      
        

    




100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 12.41it/s]


[1/10] Loss: 0.00067, Acc: 97.16


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 12.40it/s]


[2/10] Loss: 0.00063, Acc: 97.32


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 11.72it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 12.27it/s]


[9/10] Loss: 0.00044, Acc: 98.11


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:38<00:00, 12.81it/s]


[10/10] Loss: 0.00041, Acc: 98.23


AttributeError: 'Tensor' object has no attribute 'images'

#### Q5:
Please print the training and testing accuracy.

In [46]:
        
        
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
train_correct = 0
for data, target in train_loader:
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1]
    train_correct += pred.eq(target.data.view_as(pred)).cpu().sum()
print('\nTraining:  Accuracy: {}/{} ({:.0f}%)\n'.format(
     train_correct, len(train_loader.dataset),
    100. * train_correct / len(train_loader.dataset)))  

test_correct = 0
for data, target in test_loader:
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1]
    test_correct += pred.eq(target.data.view_as(pred)).cpu().sum()


print('\nTesting:  Accuracy: {}/{} ({:.0f}%)\n'.format(
     test_correct, len(test_loader.dataset),
    100. * test_correct / len(test_loader.dataset)))




Training:  Accuracy: 58939/60000 (98%)


Testing:  Accuracy: 9822/10000 (98%)

